# Market Intelligence Demo

This notebook demonstrates the full `MarketIntelligence` system, which orchestrates:

1. **RegimeAnalyzer** - Detect current market context (bull/bear/crisis/sideways)
2. **AutoPilot** - Automatically select the best analysis pipeline
3. **Explainer** - Generate actionable reports with Markdown export

The system uses real-time data from Yahoo Finance and produces end-to-end analysis with a single API call.

In [1]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

print('Setup complete.')

Setup complete.


---
## 1. Full Analysis: BTC-USD

The `analyze()` method runs the complete pipeline:
1. Fetch price data
2. Detect market regime
3. Select pipeline (conservative/aggressive/crisis/trend_following)
4. Fit model and generate forecast
5. Compile report

In [2]:
from timeseries_toolkit.intelligence import MarketIntelligence

mi = MarketIntelligence()

report = mi.analyze(['BTC-USD'], horizon='7D', verbose=True)

print('\n' + '=' * 60)
print('ANALYSIS COMPLETE')
print('=' * 60)

[1/5] Fetching data for ['BTC-USD']...


[2/5] Detecting market regime...


[3/5] Selecting pipeline...


[4/5] Running aggressive pipeline...


[5/5] Skipping driver analysis (single asset).

ANALYSIS COMPLETE


### 1.1 Report Summary

In [3]:
print('Summary:')
print(report.summary)
print()
print(f'Pipeline used: {report.pipeline_used}')
print(f'Pipeline reason: {report.pipeline_reason}')
print(f'Assets: {report.assets}')
print(f'Horizon: {report.horizon}')
print(f'Data range: {report.data_range}')
print(f'Observations: {report.observations_used}')

Summary:
MARKET REGIME: Bull (100% confidence)

BTC-USD 7D Outlook: $87,798.62 -> $87,869.70 (+0.1%)
Confidence: LOW

Key Factors:
  - Regime transition risk elevated (52%)
  - 2 of 4 diagnostic tests failed

Quality: LOW (2/4 diagnostics pass)
Risk: 52% chance of regime shift within 14 days

Pipeline used: aggressive
Pipeline reason: Data is non-stationary (ADF p > 0.05); optimal fractional d=0.50. Fractional differentiation preserves long memory while achieving stationarity (López de Prado 2018).
Assets: ['BTC-USD']
Horizon: 7D
Data range: 2025-01-29 to 2026-01-29
Observations: 366


### 1.2 Regime Detection Results

In [4]:
if report.regime:
    regime = report.regime
    print(f'Current regime:     {regime.current_regime}')
    print(f'Confidence:         {regime.confidence:.1%}')
    print(f'Days in regime:     {regime.days_in_regime}')
    print(f'Transition risk:    {regime.transition_risk:.1%}')
    print(f'\nRegime probabilities:')
    for name, prob in regime.regime_probabilities.items():
        bar = '#' * int(prob * 40)
        print(f'  {name:10s}: {prob:.1%} {bar}')
else:
    print('Regime detection was not performed.')

Current regime:     bull
Confidence:         100.0%
Days in regime:     3
Transition risk:    52.4%

Regime probabilities:
  bull      : 100.0% ########################################
  bear      : 0.0% 
  crisis    : 0.0% 


### 1.3 Forecast

In [5]:
if report.forecast is not None and not report.forecast.empty:
    print('7-Day Forecast:')
    print(report.forecast.to_string())
    
    # Direction and magnitude
    first_val = report.forecast['forecast'].iloc[0]
    last_val = report.forecast['forecast'].iloc[-1]
    current = report.current_prices.get('BTC-USD', first_val)
    pct = (last_val / current - 1) * 100
    print(f'\nCurrent price: ${current:,.0f}')
    print(f'Day 7 forecast: ${last_val:,.0f}')
    print(f'Expected change: {pct:+.2f}%')
else:
    print('No forecast available.')

7-Day Forecast:
                forecast         lower         upper
2026-01-30  87869.695084  87579.924600  88159.465567
2026-01-31  87869.695084  87459.897736  88279.492431
2026-02-01  87869.695084  87367.797884  88371.592283
2026-02-02  87869.695084  87290.154117  88449.236050
2026-02-03  87869.695084  87221.748585  88517.641582
2026-02-04  87869.695084  87159.905257  88579.484910
2026-02-05  87869.695084  87103.034448  88636.355719

Current price: $87,799
Day 7 forecast: $87,870
Expected change: +0.08%


### 1.4 Warnings and Recommendations

In [6]:
if report.warnings:
    print('Warnings:')
    for w in report.warnings:
        print(f'  - {w}')
else:
    print('No warnings.')

print()
if report.recommendations:
    print('Recommendations:')
    for r in report.recommendations:
        print(f'  - {r}')
else:
    print('No recommendations.')

Warnings:
  - Regime transition risk elevated (52%)
  - 2 of 4 diagnostic tests failed

Recommendations:
  - Trend-following strategies may be effective in bull regime


### 1.5 Quality Score

In [7]:
print(f'Quality score: {report.quality_score:.2f} / 1.00')

if report.diagnostics:
    diag = report.diagnostics
    print(f'\nDiagnostics:')
    print(f'  Residual mean: {diag.residual_mean:.6f}')
    print(f'  Residual std:  {diag.residual_std:.6f}')
    print(f'  Tests passed:  {diag.pass_count}/{diag.total_tests}')

Quality score: 0.50 / 1.00

Diagnostics:
  Residual mean: 0.000000
  Residual std:  147.844800
  Tests passed:  2/4


---
## 2. Quick Forecast: SPY

The `quick_forecast()` method skips regime detection and causal analysis for faster results.

In [8]:
spy_fc = mi.quick_forecast('SPY', horizon='7D')

print('SPY Quick Forecast (7 days):')
print(spy_fc.to_string())

SPY Quick Forecast (7 days):
              forecast       lower       upper
2026-01-29  695.396985  693.947781  696.846190
2026-01-30  695.396985  693.347501  697.446470
2026-01-31  695.396985  692.886890  697.907081
2026-02-01  695.396985  692.498577  698.295394
2026-02-02  695.396985  692.156466  698.637505
2026-02-03  695.396985  691.847174  698.946797
2026-02-04  695.396985  691.562751  699.231220


---
## 3. Regime Detection Only

Use `get_regime()` when you only need the market context without forecasting.

In [9]:
regime = mi.get_regime(['SPY'])

print('SPY Market Regime:')
print(f'  Current:        {regime.current_regime}')
print(f'  Confidence:     {regime.confidence:.1%}')
print(f'  Days in regime: {regime.days_in_regime}')
print(f'  Transition risk: {regime.transition_risk:.1%}')

SPY Market Regime:
  Current:        bull
  Confidence:     86.4%
  Days in regime: 6
  Transition risk: 22.3%


---
## 4. Multi-Asset Comparison

Compare BTC and SPY side-by-side with correlation analysis.

In [10]:
comparison = mi.compare_assets(['BTC-USD', 'SPY'], horizon='7D')

print('Asset Comparison: BTC-USD vs SPY')
print()

if comparison.correlations is not None:
    print('Return Correlations:')
    print(comparison.correlations.to_string())
    print()

if comparison.relative_strength is not None:
    print('30-Day Relative Strength:')
    print(comparison.relative_strength.to_string())
    print()

for asset, rpt in comparison.individual_reports.items():
    print(f'\n--- {asset} ---')
    print(f'  Pipeline: {rpt.pipeline_used}')
    if rpt.regime:
        print(f'  Regime:   {rpt.regime.current_regime} ({rpt.regime.confidence:.0%})')
    if rpt.forecast is not None and not rpt.forecast.empty:
        last_fc = rpt.forecast["forecast"].iloc[-1]
        print(f'  7D forecast: ${last_fc:,.2f}')

Asset Comparison: BTC-USD vs SPY

Return Correlations:
          BTC-USD       SPY
BTC-USD  1.000000  0.416875
SPY      0.416875  1.000000

30-Day Relative Strength:
         cumulative_return
BTC-USD           0.120445
SPY               0.040084


--- BTC-USD ---
  Pipeline: aggressive
  Regime:   bull (100%)
  7D forecast: $87,869.70

--- SPY ---
  Pipeline: aggressive
  Regime:   bull (91%)
  7D forecast: $695.40


---
## 5. Export to Markdown

Reports can be exported as structured Markdown for documentation or sharing.

In [11]:
md = report.to_markdown()

print('Markdown Report (first 2000 characters):')
print('-' * 60)
print(md[:2000])
if len(md) > 2000:
    print(f'\n... ({len(md)} total characters)')

Markdown Report (first 2000 characters):
------------------------------------------------------------
# Market Intelligence Report

**Generated**: 2026-01-29 10:32:26 UTC  
**Assets**: BTC-USD  
**Horizon**: 7D  
**Quality Score**: 0.50 (LOW)

---

## Executive Summary

MARKET REGIME: Bull (100% confidence)

BTC-USD 7D Outlook: $87,798.62 -> $87,869.70 (+0.1%)
Confidence: LOW

Key Factors:
  - Regime transition risk elevated (52%)
  - 2 of 4 diagnostic tests failed

Quality: LOW (2/4 diagnostics pass)
Risk: 52% chance of regime shift within 14 days

---

## Current Regime Analysis

| Metric | Value |
|--------|-------|
| Current Regime | Bull |
| Confidence | 100% |
| Days in Regime | 3 |
| Transition Risk (14d) | 52% |

### Regime Probabilities

| Regime | Probability |
|--------|-------------|
| Bull | 100% |
| Bear | 0% |
| Crisis | 0% |

---

## Forecast Details

### BTC-USD

| Date | Forecast | Lower (95%) | Upper (95%) |
|------|----------|-------------|-------------|
| 2026-01-3

### 5.1 Export to Dict and DataFrame

In [12]:
report_dict = report.to_dict()
print('Report as dict - keys:')
for key in report_dict:
    print(f'  {key}')

print()
report_df = report.to_dataframe()
print(f'Report as DataFrame: {report_df.shape}')
report_df

Report as dict - keys:
  timestamp
  assets
  horizon
  quality_score
  pipeline_used
  pipeline_reason
  summary
  warnings
  recommendations
  regime
  forecast
  diagnostics

Report as DataFrame: (7, 3)


,forecast,lower,upper
2026-01-30,87869.695084,87579.924600,88159.465567
2026-01-31,87869.695084,87459.897736,88279.492431
2026-02-01,87869.695084,87367.797884,88371.592283
2026-02-02,87869.695084,87290.154117,88449.236050
2026-02-03,87869.695084,87221.748585,88517.641582
2026-02-04,87869.695084,87159.905257,88579.484910
2026-02-05,87869.695084,87103.034448,88636.355719


---
## 6. Using Individual Layers

The three layers can be used independently for more control.

### 6.1 RegimeAnalyzer Standalone

In [13]:
from timeseries_toolkit.intelligence import RegimeAnalyzer
from timeseries_toolkit.data_sources import CryptoDataLoader

analyzer = RegimeAnalyzer()
crypto = CryptoDataLoader()
btc_df = crypto.get_prices(['BTC-USD'], period='1y')
btc_close = btc_df[[c for c in btc_df.columns if 'Close' in c or 'close' in c][0]]
prices = pd.DataFrame({'BTC-USD': btc_close})

result = analyzer.detect(prices)

print(f'Regime:          {result.current_regime}')
print(f'Confidence:      {result.confidence:.1%}')
print(f'Days in regime:  {result.days_in_regime}')
print(f'Transition risk: {result.transition_risk:.1%}')

Regime:          bull


Confidence:      100.0%
Days in regime:  3
Transition risk: 52.4%


### 6.2 AutoPilot Standalone

In [14]:
from timeseries_toolkit.intelligence import AutoPilot

autopilot = AutoPilot()
pipeline, reason = autopilot.select_pipeline(
    data=btc_close,
    regime=result.current_regime,
    horizon=7
)

print(f'Selected pipeline: {pipeline.name}')
print(f'Reason: {reason}')
print(f'Steps: {pipeline.steps}')

Selected pipeline: aggressive
Reason: Data is non-stationary (ADF p > 0.05); optimal fractional d=0.50. Fractional differentiation preserves long memory while achieving stationarity (López de Prado 2018).
Steps: [('fracdiff', None), ('model', <timeseries_toolkit.models.kalman.AutoKalmanFilter object at 0x000001BC84894080>)]


### 6.3 Pipeline Fit and Predict

In [15]:
pipeline.fit(btc_close)
forecast_df = pipeline.predict(horizon=7, confidence_level=0.95)

print('BTC-USD 7-Day Pipeline Forecast:')
print(forecast_df.to_string())

diag = pipeline.get_diagnostics()
print(f'\nDiagnostics:')
print(f'  Residual mean: {diag.residual_mean:.4f}')
print(f'  Residual std:  {diag.residual_std:.4f}')
print(f'  Tests passed:  {diag.pass_count}/{diag.total_tests}')

BTC-USD 7-Day Pipeline Forecast:
                forecast         lower         upper
2026-01-30  87869.695084  87579.924600  88159.465567
2026-01-31  87869.695084  87459.897736  88279.492431
2026-02-01  87869.695084  87367.797884  88371.592283
2026-02-02  87869.695084  87290.154117  88449.236050
2026-02-03  87869.695084  87221.748585  88517.641582
2026-02-04  87869.695084  87159.905257  88579.484910
2026-02-05  87869.695084  87103.034448  88636.355719

Diagnostics:
  Residual mean: 0.0000
  Residual std:  147.8448
  Tests passed:  2/4


---
## Summary

| API | Method | Use Case |
|-----|--------|----------|
| `MarketIntelligence` | `analyze()` | Full analysis with regime + forecast + report |
| `MarketIntelligence` | `quick_forecast()` | Fast prediction only |
| `MarketIntelligence` | `get_regime()` | Market context detection |
| `MarketIntelligence` | `compare_assets()` | Multi-asset comparison |
| `IntelligenceReport` | `to_markdown()` | Export as Markdown |
| `IntelligenceReport` | `to_dict()` / `to_dataframe()` | Programmatic access |
| `RegimeAnalyzer` | `detect()` | Standalone regime detection |
| `AutoPilot` | `select_pipeline()` | Standalone pipeline selection |
| `Pipeline` | `fit()` / `predict()` | Direct pipeline execution |